# Formalism

Consider a 4-level system here. Energies of level $\left|1\right>$, $\left|2\right>$ and $\left|3\right>$ are in increasing order, and there one laser coupling state $\left|1\right>$ to $\left|2\right>$, and another laser coupling state $\left|2\right>$ to $\left|3\right>$. State $\left|2\right>$ can decay to state $\left|1\right>$. State $\left|3\right>$ can decay to state $\left|2\right>$ and $\left|4\right>$. Now we want to solve the population evolution using master equations.

Master equations can be written as 
$$\dot{\rho}=\frac{-i}{\hbar}\left[H,\ \rho\right]+\sum_i\Gamma_i\left(L_i\rho L_i^\dagger-\frac{1}{2}L_i^\dagger L_i\rho-\frac{1}{2}\rho L_i^\dagger L_i\right).$$ 
Under a unitary transformation (or rotating frame $\rho\rightarrow U\rho U^\dagger$), all the operators transform in the same way and the master equation can be re-written as
$$\dot{\rho}=-i\left[H/\hbar+i\dot{U}U^\dagger,\ \rho\right]+\sum_i\Gamma_i\left(L_i\rho L_i^\dagger-\frac{1}{2}L_i^\dagger L_i\rho-\frac{1}{2}\rho L_i^\dagger L_i\right).$$ 

Here we use $\omega_i$ for energy of level $\left|i\right>$, and $\Gamma_{ij}$ for decay rate from level $\left|i\right>$ and $\left|j\right>$. We use $\omega_{ij}$, $\Omega_{ij}$ and $\Delta_{ij}$ for frequency, Rabi frequency and detuning respectively of the laser that couples level $\left|i\right>$ and $\left|j\right>$ (assume one laser only couples one pair of states).

In [1]:
import numpy as np
import sympy
from sympy.physics.quantum.dagger import Dagger

omega_1, omega_2, omega_3, omega_4 = sympy.symbols('omega_1 omega_2 omega_3 omega_4', real=True) # energy of states
Omega_12, Omega_23 = sympy.symbols('Omega_12 Omega_23', real=True) # Rabi frequency
omega_12, omega_23 = sympy.symbols('omega_12 omega_23', real=True) # laser frequency
Delta_12, Delta_23 = sympy.symbols('Delta_12 Delta_23', real=True) # laser frequency
t = sympy.symbols('t', positive=True) # time

rho_11, rho_12, rho_13, rho_14 = sympy.symbols('rho_11 rho_12 rho_13 rho_14') # density matrix elements
rho_21, rho_22, rho_23, rho_24 = sympy.symbols('rho_21 rho_22 rho_23 rho_24') # density matrix elements
rho_31, rho_32, rho_33, rho_34 = sympy.symbols('rho_31 rho_32 rho_33 rho_34') # density matrix elements
rho_41, rho_42, rho_43, rho_44 = sympy.symbols('rho_41 rho_42 rho_43 rho_44') # density matrix elements

In [2]:
Ham = sympy.Matrix([[omega_1, Omega_12*sympy.exp(sympy.I*omega_12*t)/2, 0, 0], 
                    [Omega_12*sympy.exp(-sympy.I*omega_12*t)/2, omega_2, Omega_23*sympy.exp(sympy.I*omega_23*t)/2, 0],
                    [0, Omega_23*sympy.exp(-sympy.I*omega_23*t)/2, omega_3, 0],
                    [0, 0, 0, omega_4]]) # Hamiltonian
                    
print("Hamiltonian in lab frame:")
Ham

Hamiltonian in lab frame:


Matrix([
[                      omega_1,  Omega_12*exp(I*omega_12*t)/2,                            0,       0],
[Omega_12*exp(-I*omega_12*t)/2,                       omega_2, Omega_23*exp(I*omega_23*t)/2,       0],
[                            0, Omega_23*exp(-I*omega_23*t)/2,                      omega_3,       0],
[                            0,                             0,                            0, omega_4]])

In [3]:
Uni = sympy.Matrix([[1, 0, 0, 0],
                    [0, sympy.exp(sympy.I*omega_12*t), 0, 0],
                    [0, 0, sympy.exp(sympy.I*(omega_12+omega_23)*t), 0],
                    [0, 0, 0, sympy.exp(sympy.I*(omega_4-omega_1)*t)]]) # Unitary matrix 
                    
print("Unitary transformation:")
Uni

Unitary transformation:


Matrix([
[1,                 0,                              0,                             0],
[0, exp(I*omega_12*t),                              0,                             0],
[0,                 0, exp(I*t*(omega_12 + omega_23)),                             0],
[0,                 0,                              0, exp(I*t*(-omega_1 + omega_4))]])

In [4]:
Ham_rot = Uni.multiply(Ham).multiply(Dagger(Uni)) # Unitary transformation
Ham_rot += sympy.I*sympy.diff(Uni, t).multiply(Dagger(Uni)) # effective Hamiltonian
Ham_rot -= omega_1*sympy.eye(4) # shift energy
Ham_rot = sympy.simplify(Ham_rot)

print("Effective Hamiltonian in ratating frame")
Ham_rot

Effective Hamiltonian in ratating frame


Matrix([
[         0,                    Omega_12/2,                                        0, 0],
[Omega_12/2, -omega_1 - omega_12 + omega_2,                               Omega_23/2, 0],
[         0,                    Omega_23/2, -omega_1 - omega_12 - omega_23 + omega_3, 0],
[         0,                             0,                                        0, 0]])

In [5]:
Ham_rot_2 = sympy.Matrix([[0, Omega_12/2, 0, 0], 
                    [Omega_12/2, -Delta_23, Omega_23/2, 0],
                    [0, Omega_23/2, -Delta_23-Delta_12, 0],
                    [0, 0, 0, 0]]) # Hamiltonian

print("Re-write (effective) Hamiltonian in rotating frame:")
Ham_rot_2

Re-write (effective) Hamiltonian in rotating frame:


Matrix([
[         0, Omega_12/2,                    0, 0],
[Omega_12/2,  -Delta_23,           Omega_23/2, 0],
[         0, Omega_23/2, -Delta_12 - Delta_23, 0],
[         0,          0,                    0, 0]])

In [6]:
den = sympy.Matrix([[rho_11, rho_12, rho_13, rho_14], 
                    [rho_21, rho_22, rho_23, rho_24],
                    [rho_31, rho_32, rho_33, rho_34],
                    [rho_41, rho_42, rho_43, rho_44]]) # Density matrix

print("Density matrix (in rotating frame):")
den

Density matrix (in rotating frame):


Matrix([
[rho_11, rho_12, rho_13, rho_14],
[rho_21, rho_22, rho_23, rho_24],
[rho_31, rho_32, rho_33, rho_34],
[rho_41, rho_42, rho_43, rho_44]])

In [7]:
L_21 = sympy.Matrix([[0, 1, 0, 0], 
                    [0, 0, 0, 0],
                    [0, 0, 0, 0],
                    [0, 0, 0, 0]]) # Lindblad operator for decay from state |2> to |1>
L_21_rot = Uni.multiply(L_21).multiply(Dagger(Uni))

print("Lindblad operator L_12 in rotating frame:")
L_21_rot

Lindblad operator L_12 in rotating frame:


Matrix([
[0, exp(-I*omega_12*t), 0, 0],
[0,                  0, 0, 0],
[0,                  0, 0, 0],
[0,                  0, 0, 0]])

In [8]:
Lindblad_21 = L_21_rot.multiply(den).multiply(Dagger(L_21_rot))-Dagger(L_21_rot).multiply(L_21_rot).multiply(den)/2-den.multiply(Dagger(L_21_rot)).multiply(L_21_rot)/2

print("Lindblad terms from L_21:")
Lindblad_21

Lindblad terms from L_21:


Matrix([
[   rho_22, -rho_12/2,         0,         0],
[-rho_21/2,   -rho_22, -rho_23/2, -rho_24/2],
[        0, -rho_32/2,         0,         0],
[        0, -rho_42/2,         0,         0]])

In [9]:
L_32 = sympy.Matrix([[0, 0, 0, 0], 
                    [0, 0, 1, 0],
                    [0, 0, 0, 0],
                    [0, 0, 0, 0]]) # Lindblad operator for decay from state |3> to |2>
L_32_rot = Uni.multiply(L_32).multiply(Dagger(Uni))

print("Lindblad operator L_32 in rotating frame:")
L_32_rot

Lindblad operator L_32 in rotating frame:


Matrix([
[0, 0,                                                 0, 0],
[0, 0, exp(I*omega_12*t)*exp(-I*t*(omega_12 + omega_23)), 0],
[0, 0,                                                 0, 0],
[0, 0,                                                 0, 0]])

In [10]:
Lindblad_32 = L_32_rot.multiply(den).multiply(Dagger(L_32_rot))-Dagger(L_32_rot).multiply(L_32_rot).multiply(den)/2-den.multiply(Dagger(L_32_rot)).multiply(L_32_rot)/2

print("Lindblad terms from L_32:")
Lindblad_32

Lindblad terms from L_32:


Matrix([
[        0,         0, -rho_13/2,         0],
[        0,    rho_33, -rho_23/2,         0],
[-rho_31/2, -rho_32/2,   -rho_33, -rho_34/2],
[        0,         0, -rho_43/2,         0]])

In [11]:
L_34 = sympy.Matrix([[0, 0, 0, 0], 
                    [0, 0, 0, 0],
                    [0, 0, 0, 0],
                    [0, 0, 1, 0]]) # Lindblad operator for decay from state |3> to |2>
L_34_rot = Uni.multiply(L_34).multiply(Dagger(Uni))

print("Lindblad operator L_34 in rotating frame:")
L_34_rot

Lindblad operator L_34 in rotating frame:


Matrix([
[0, 0,                                                             0, 0],
[0, 0,                                                             0, 0],
[0, 0,                                                             0, 0],
[0, 0, exp(I*t*(-omega_1 + omega_4))*exp(-I*t*(omega_12 + omega_23)), 0]])

In [12]:
Lindblad_34 = L_34_rot.multiply(den).multiply(Dagger(L_34_rot))-Dagger(L_34_rot).multiply(L_34_rot).multiply(den)/2-den.multiply(Dagger(L_34_rot)).multiply(L_34_rot)/2

print("Lindblad terms from L_34:")
Lindblad_34

Lindblad terms from L_34:


Matrix([
[        0,         0, -rho_13/2,         0],
[        0,         0, -rho_23/2,         0],
[-rho_31/2, -rho_32/2,   -rho_33, -rho_34/2],
[        0,         0, -rho_43/2,    rho_33]])